In [1]:
import tensorflow as tf

In [6]:
# TensorFlow Hello World (with Eager Execution)
a = tf.constant(10)  # create tensor for a and b
b = tf.constant(32)
print(a+b)


Tensor("add_1:0", shape=(), dtype=int32)


In [7]:
# TensorFlow Hello World (with Graph Execution)
# this is to disable eager execution mode
tf.compat.v1.disable_eager_execution()

a = tf.constant(10)  # create tensor for a and b
b = tf.constant(32)
sess = tf.compat.v1.Session()
print(sess.run(a+b))


42


In [2]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(12, input_dim=8, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='relu')
])

# Compile Model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()

# from keras.utils.vis_utils import plot_model
# plot_model(model, show_shapes=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                108       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


2021-11-04 12:17:06.987512: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
import numpy as np
tf.compat.v1.disable_eager_execution()

x_data = np.random.rand(100).astype(np.float32)
y_data = x_data * 0.1 + 0.3

W = tf.Variable(tf.random.uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))

y = W * x_data + b

loss = tf.reduce_mean(tf.square(y - y_data))
# optimizer = tf.keras.optimizers.SGD(0.5)
optimizer = tf.compat.v1.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

init = tf.compat.v1.initialize_all_variables()

sess = tf.compat.v1.Session()
sess.run(init)

for step in range(201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))

0 [0.01859346] [0.528876]
20 [0.0634169] [0.32180855]
40 [0.09167339] [0.30496383]
60 [0.09810479] [0.30112982]
80 [0.09956866] [0.30025715]
100 [0.09990183] [0.30005854]
120 [0.09997767] [0.30001333]
140 [0.09999491] [0.30000305]
160 [0.09999884] [0.3000007]
180 [0.09999973] [0.30000016]
200 [0.09999992] [0.30000007]


In [11]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris()
x, y = iris.data, iris.target

x_train,x_test, y_train,  y_test = train_test_split(x, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

feature_col = [
    tf.feature_column.numeric_column("x", shape=4)
]
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_col,
    hidden_units=[10, 20, 32, 10],
    n_classes=3
)

train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(x={"x" : x_train},
                                                    y= y_train,
                                                    batch_size=6,
                                                    shuffle=False,
                                                    num_epochs=150
)
classifier.train(input_fn=train_input_fn)

test_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"x" : x_test},
    y=y_test,
    batch_size=1,
    shuffle=False
)
scores = classifier.evaluate(input_fn=test_input_fn)
print(f"Accuracy: {scores['accuracy']} \tLoss: {scores['loss']}")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/jr/017d8q8145z52wbhxmqqxxkc0000gn/T/tmpx9ukrea5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INF

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-11-04T12:40:09
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/jr/017d8q8145z52wbhxmqqxxkc0000gn/T/tmpx9ukrea5/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.32485s
INFO:tensorflow:Finished evaluation at 2021-11-04-12:40:09
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.96666664, average_loss = 0.6207546, global_step = 3000, loss = 0.6207546
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3000: /var/folders/jr/017d8q8145z52wbhxmqqxxkc0000gn/T/tmpx9ukrea5/model.ckpt-3000
Accuracy: 0.9666666388511658 	Loss: 0.620754599571228


In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_dim=8, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='rmsprop',
    loss= "categorical_crossentropy",
    metrics=['accuracy']
)

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                576       
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                650       
Total params: 5,386
Trainable params: 5,386
Non-trainable params: 0
_________________________________________________________________


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
iris = pd.read_csv("iris.csv")

x = np.array(iris.drop("Class", axis=1))
y = np.array(iris["Class"])

# train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Scale dataset
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

# create model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_dim=4, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(
    optimizer='rmsprop',
    loss= "binary_crossentropy",
    metrics=['accuracy']
)

# Train model
model.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test))
# calculate predictions
predictions = model.predict(x_test)

# round predictions
rounded = [round(x[0]) for x in predictions]
print(rounded)


Train on 80 samples, validate on 20 samples
Epoch 1/100
80/80 [==============================] - 0s 2ms/sample - loss: 0.5377 - accuracy: 0.7625 - val_loss: 0.4391 - val_accuracy: 1.0000
Epoch 2/100
80/80 [==============================] - 0s 160us/sample - loss: 0.3928 - accuracy: 0.9875 - val_loss: 0.3343 - val_accuracy: 1.0000
Epoch 3/100
80/80 [==============================] - 0s 169us/sample - loss: 0.3014 - accuracy: 1.0000 - val_loss: 0.2532 - val_accuracy: 1.0000
Epoch 4/100
80/80 [==============================] - 0s 176us/sample - loss: 0.2298 - accuracy: 1.0000 - val_loss: 0.1859 - val_accuracy: 1.0000
Epoch 5/100
80/80 [==============================] - 0s 183us/sample - loss: 0.1713 - accuracy: 1.0000 - val_loss: 0.1346 - val_accuracy: 1.0000
Epoch 6/100
80/80 [==============================] - 0s 133us/sample - loss: 0.1245 - accuracy: 1.0000 - val_loss: 0.0971 - val_accuracy: 1.0000
Epoch 7/100
80/80 [==============================] - 0s 217us/sample - loss: 0.0880 - ac

/Users/angks/opt/miniconda3/envs/tf2.6/lib/python3.9/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Epoch 13/100
80/80 [==============================] - 0s 150us/sample - loss: 0.0070 - accuracy: 1.0000 - val_loss: 0.0137 - val_accuracy: 1.0000
Epoch 14/100
80/80 [==============================] - 0s 187us/sample - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0115 - val_accuracy: 1.0000
Epoch 15/100
80/80 [==============================] - 0s 199us/sample - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0105 - val_accuracy: 1.0000
Epoch 16/100
80/80 [==============================] - 0s 183us/sample - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.0098 - val_accuracy: 1.0000
Epoch 17/100
80/80 [==============================] - 0s 160us/sample - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0088 - val_accuracy: 1.0000
Epoch 18/100
80/80 [==============================] - 0s 121us/sample - loss: 7.1910e-04 - accuracy: 1.0000 - val_loss: 0.0080 - val_accuracy: 1.0000
Epoch 19/100
80/80 [==============================] - 0s 129us/sample - loss: 4.5825e-04 - accuracy: 1.0000 - val_loss: 

[0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1]
